In [21]:
#Import necessary libraries
import pandas as pd
import numpy as np

#Loads the dataset (note: If the csv is not stored in the same file path as this notebook, adjust the path in the code below to match the correct csv filepath.)
df = pd.read_csv('train.csv')

# Renames the columns to be more Power BI friendly (e.g., removes spaces or special characters)
df.columns = df.columns.str.replace(' ', '_')
print("Data types after conversion:")
df.info()

Data types after conversion:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row_ID         9800 non-null   int64  
 1   Order_ID       9800 non-null   object 
 2   Order_Date     9800 non-null   object 
 3   Ship_Date      9800 non-null   object 
 4   Ship_Mode      9800 non-null   object 
 5   Customer_ID    9800 non-null   object 
 6   Customer_Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal_Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product_ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product_Name   9800 non-null   object 
 17  Sales          9800 non

In [22]:
# See the first 5 rows to understand the structure
print("First 5 Rows:")
print(df.head())
print("\n" + "="*50 + "\n")

# Get a summary of the dataset (columns, data types, non-null values)
print("DataFrame Info:")
df.info()
print("\n" + "="*50 + "\n")

# Check for missing values in each column
print("Missing Values per Column:")
print(df.isnull().sum())
print("\n" + "="*50 + "\n")

# Check for duplicate rows
print(f"Number of duplicate rows: {df.duplicated().sum()}")

First 5 Rows:
   Row_ID        Order_ID  Order_Date   Ship_Date       Ship_Mode Customer_ID  \
0       1  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
1       2  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
2       3  CA-2017-138688  12/06/2017  16/06/2017    Second Class    DV-13045   
3       4  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   
4       5  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   

     Customer_Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

   Postal_Code Region       Product_

In [23]:
# Create a new DataFrame containing only the rows where Postal Code is null
missing_postal_code_rows = df[df['Postal_Code'].isnull()]

# Display these rows to see if we can find a pattern
print(missing_postal_code_rows)

      Row_ID        Order_ID  Order_Date   Ship_Date       Ship_Mode  \
2234    2235  CA-2018-104066  05/12/2018  10/12/2018  Standard Class   
5274    5275  CA-2016-162887  07/11/2016  09/11/2016    Second Class   
8798    8799  US-2017-150140  06/04/2017  10/04/2017  Standard Class   
9146    9147  US-2017-165505  23/01/2017  27/01/2017  Standard Class   
9147    9148  US-2017-165505  23/01/2017  27/01/2017  Standard Class   
9148    9149  US-2017-165505  23/01/2017  27/01/2017  Standard Class   
9386    9387  US-2018-127292  19/01/2018  23/01/2018  Standard Class   
9387    9388  US-2018-127292  19/01/2018  23/01/2018  Standard Class   
9388    9389  US-2018-127292  19/01/2018  23/01/2018  Standard Class   
9389    9390  US-2018-127292  19/01/2018  23/01/2018  Standard Class   
9741    9742  CA-2016-117086  08/11/2016  12/11/2016  Standard Class   

     Customer_ID     Customer_Name      Segment        Country        City  \
2234    QJ-19255      Quincy Jones    Corporate  United S

Based on a brief observation of the missing Postal Code values, we can see that all the missing postal codes are for the city of Burlington in the United States. This means, to handle the missing values, we simply need to replace the missing values with the correct primary postal code for Burlington which is: 05401.

Burlington has a total of seven postal codes, of which, 05401 is considered the main postal code. However, this isn't an issue because our analysis will be at the state, regional, and city level and not a deep dive at the postal level. This means that even if we aren't 100% right, the impact of not having the exact postal code is negligable.

In [25]:
# Define the correct postal code for Burlington, Vermont
burlington_postal_code = 5401 # Use the number 5401, not the string '05401'

# Use .loc to find rows where 'City' is 'Burlington' and fill the 'Postal Code'
df.loc[df['City'] == 'Burlington', 'Postal_Code'] = df.loc[df['City'] == 'Burlington', 'Postal_Code'].fillna(burlington_postal_code)

In [26]:
# Verify that there are no more missing values in the 'Postal Code' column
print("Missing Values After Fix:")
print(df.isnull().sum())

Missing Values After Fix:
Row_ID           0
Order_ID         0
Order_Date       0
Ship_Date        0
Ship_Mode        0
Customer_ID      0
Customer_Name    0
Segment          0
Country          0
City             0
State            0
Postal_Code      0
Region           0
Product_ID       0
Category         0
Sub-Category     0
Product_Name     0
Sales            0
dtype: int64


In [27]:
# Convert 'Order Date' and 'Ship Date' to datetime objects
df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%d/%m/%Y')
df['Ship_Date'] = pd.to_datetime(df['Ship_Date'], format='%d/%m/%Y')

In [28]:
# Creates 'Days to Ship'
df['Days_to_Ship'] = (df['Ship_Date'] - df['Order_Date']).dt.days

# Extracts 'Order Year' and 'Order Month Name'
df['Order_Year'] = df['Order_Date'].dt.year
df['Order_Month_Name'] = df['Order_Date'].dt.month_name()

# Check the result
print(df[['Order_Date', 'Ship_Date', 'Days_to_Ship']].head())

  Order_Date  Ship_Date  Days_to_Ship
0 2017-11-08 2017-11-11             3
1 2017-11-08 2017-11-11             3
2 2017-06-12 2017-06-16             4
3 2016-10-11 2016-10-18             7
4 2016-10-11 2016-10-18             7


In [29]:
# Define the columns we need to answer our business questions
columns_to_keep = [
    'Order_ID',
    'Order_Date',
    'Ship_Date',
    'Ship_Mode',
    'Customer_ID',
    'Customer_Name',
    'Segment',
    'City',
    'State',
    'Postal_Code',
    'Region',
    'Product_ID',
    "Product_Name",
    'Category',
    'Sub-Category',
    'Sales',
    'Days_to_Ship',
    'Order_Year',
    'Order_Month_Name'
]

# Create the final, lean DataFrame
df_final = df[columns_to_keep]

# Save the final DataFrame to CSV
df_final.to_csv('cleaned_superstore_data.csv', index=False)

print("\nFinal dataset created with the following columns:")
print(df_final.columns.tolist())


Final dataset created with the following columns:
['Order_ID', 'Order_Date', 'Ship_Date', 'Ship_Mode', 'Customer_ID', 'Customer_Name', 'Segment', 'City', 'State', 'Postal_Code', 'Region', 'Product_ID', 'Product_Name', 'Category', 'Sub-Category', 'Sales', 'Days_to_Ship', 'Order_Year', 'Order_Month_Name']


In [30]:
df = pd.read_csv('cleaned_superstore_data.csv')

In [12]:
# See the first 5 rows to understand the structure
print("First 5 Rows:")
print(df.head())
print("\n" + "="*50 + "\n")

# Get a summary of the dataset (columns, data types, non-null values)
print("DataFrame Info:")
df.info()
print("\n" + "="*50 + "\n")

# Check for missing values in each column
print("Missing Values per Column:")
print(df.isnull().sum())
print("\n" + "="*50 + "\n")

# Check for duplicate rows
print(f"Number of duplicate rows: {df.duplicated().sum()}")

First 5 Rows:
         Order_ID  Order_Date   Ship_Date       Ship_Mode Customer_ID  \
0  CA-2017-152156  2017-11-08  2017-11-11    Second Class    CG-12520   
1  CA-2017-152156  2017-11-08  2017-11-11    Second Class    CG-12520   
2  CA-2017-138688  2017-06-12  2017-06-16    Second Class    DV-13045   
3  US-2016-108966  2016-10-11  2016-10-18  Standard Class    SO-20335   
4  US-2016-108966  2016-10-11  2016-10-18  Standard Class    SO-20335   

     Customer_Name    Segment             City       State  Postal_Code  \
0      Claire Gute   Consumer        Henderson    Kentucky      42420.0   
1      Claire Gute   Consumer        Henderson    Kentucky      42420.0   
2  Darrin Van Huff  Corporate      Los Angeles  California      90036.0   
3   Sean O'Donnell   Consumer  Fort Lauderdale     Florida      33311.0   
4   Sean O'Donnell   Consumer  Fort Lauderdale     Florida      33311.0   

  Region       Product_ID                                       Product_Name  \
0  South  FUR-BO

In [ ]:
#Customer Analaysis - Customer Segmentation

#Types of customers
Customer_types = df['Segment'].unique()
print(Customer_types)

#Amount of each customer type
number_of_customers = df['Segment'].value_counts().reset_index()

number_of_customers = number_of_customers.rename(columns={'Segment' : 'Customer Type', 'count': 'Total Customers'})

print(number_of_customers)

['Consumer' 'Corporate' 'Home Office']
  Customer Type  Total Customers
0      Consumer             5101
1     Corporate             2953
2   Home Office             1746
